# Event Module

Copyright 2022 Michael George (AKA Logiqx).

This file is part of [sse-results](https://github.com/Logiqx/sse-results) and is distributed under the terms of the GNU General Public License.

sse-results is free software: you can redistribute it and/or modify it under the terms of the GNU General Public License as published by the Free Software Foundation, either version 3 of the License, or (at your option) any later version.

sse-results is distributed in the hope that it will be useful, but WITHOUT ANY WARRANTY; without even the implied warranty of MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE. See the GNU General Public License for more details.

You should have received a copy of the GNU General Public License along with sse-results. If not, see <https://www.gnu.org/licenses/>.

## Initialisation

Basic approach to determine the project directory

In [1]:
import os
import glob

import csv
import json

import math
import unittest

from common import testExit, projdir

from name import Name
from fuzzy import FuzzyMatch
from entrant import Entrant
from filter import Filter
from reports import Reports

from period import Period
from session import Session

from constants import *

## Event Class

Class to manage events

In [2]:
class Event(Period):
    def __init__(self, path, appConfig, existingNames={}, verbosity=1):
        
        super().__init__(verbosity=verbosity)

        self.path = path
        self.year = int(os.path.basename(path))

        self.appConfig = appConfig

        self.existingNames = existingNames
        self.craftTypes = {}
        
        self.motions = {}
        self.sessions = {}
        
        self.initialised = False


    def checkName(self, nameObj):
        '''Check if name is a potential typo (handles single name)'''

        name = nameObj.name

        if name in self.existingNames:
            self.existingNames[name].years.append(self.year)
        else:
            switchedNameObj = Name('{} {}'.format(nameObj.lastName, nameObj.firstName))

            for existingName in self.existingNames:
                existingNameObj = self.existingNames[existingName]
                if self.fuzzyMatch.matchNameObjects(nameObj, existingNameObj) is True:
                    self.logWarning("Similar names - '{}' vs '{}' {}".format(
                        name, existingNameObj.name, existingNameObj.years))
                if self.fuzzyMatch.matchNameObjects(switchedNameObj, existingNameObj) is True:
                    self.logWarning("Similar names - '{}' vs '{}' {}".format(
                        name, existingNameObj.name, existingNameObj.years))

            self.existingNames[name] = nameObj
            nameObj.years = [self.year]
            nameObj.bests = {}


    def checkNames(self, entrant):
        '''Check if name is a potential typo (handles joint names)'''

        nameObj = entrant.name

        # Joint names use a plus sign between the names
        if '+' in nameObj.name:
            for name in [name.strip() for name in nameObj.name.split('+')]:
                nameObj = Name(name)
                self.checkName(nameObj)
        else:
            self.checkName(nameObj)

        name = entrant.getName()
        craftType = entrant.getCraftType()

        if name != 'TBC' and craftType != 'TBC':
            if craftType not in self.craftTypes:
                self.craftTypes[craftType] = {name: entrant}
            else:
                if name not in self.craftTypes[craftType]:
                    self.craftTypes[craftType][name] = entrant
                else:
                    if craftType != 'Boat':
                        self.logWarning("Duplicate name {} ({}) in {}".format(name, craftType, self.year))

        # Copy PB from existing name
        if name in self.existingNames and craftType in self.existingNames[name].bests:
            entrant.entrantDict[T_PB] = self.existingNames[name].bests[craftType]
        else:
            entrant.entrantDict[T_PB] = 0


    def loadConfig(self):
        '''Read app config from JSON'''

        filename = os.path.join(self.path, CONFIG_DIR, EVENT_CONFIG)
        with open(filename, 'r', encoding='utf-8') as f:
            jsonTxt = f.read()
            try:
                self.eventConfig = json.loads(jsonTxt)
            except:
                self.logError('Could not parse {}'.format(filename))
                raise

        if 'Comment' in self.eventConfig:
            self.comment = self.eventConfig['Comment']
        else:
            self.comment = None


    def loadMotions(self):
        '''Load motions from CSV'''

        self.motions = {}

        csvPath = os.path.join(self.path, CONFIG_DIR, MOTIONS_CSV)
        if os.path.exists(csvPath):
            with open(csvPath, 'r', encoding='utf-8') as f:
                csvReader = csv.DictReader(f)

                for motionUser in csvReader:
                    entrantId = int(motionUser['Entrant ID'])

                    if entrantId not in self.motions:
                        self.motions[entrantId] = []

                    self.motions[entrantId].append(motionUser)


    def loadEntrants(self):
        '''Read entrants from JSON'''

        csvPath = os.path.join(self.path, CONFIG_DIR, ENTRANTS_CSV)
        
        with open(csvPath, 'r', encoding='utf-8') as f:
            csvReader = csv.reader(f)
            headers = next(csvReader)

            for values in csvReader:
                if ''.join(values).strip():
                    entrant = Entrant(self.eventConfig, headers, values, motions=self.motions, verbosity=self.verbosity)
                    if entrant.getValue('ID') not in self.entrants:
                        self.entrants[entrant.getValue('ID')] = entrant

                        self.checkNames(entrant)
                    else:
                        raise ValueError('Duplicate entrant ID "{}"'.format(entrant.getValue('ID')))

        self.entrantsFilter = Filter(self.entrants)


    def loadReports(self):
        '''Read reports from JSON and (indirectly) pre-apply the entrant filters'''

        self.reports = Reports(self, verbosity=self.verbosity)

        self.reports.loadReports()
        

    def indexNames(self):
        '''Create name indices for entrants'''

        for entrantId in self.entrants:
            entrant = self.entrants[entrantId]
            name = entrant.getName()

            if name in self.names:
                self.names[name].append(entrant)
            else:
                self.names[name] = [entrant]
        

    def indexSails(self):
        '''Create sail number indices for entrants'''

        for entrantId in self.entrants:
            entrant = self.entrants[entrantId]

            for sailNo in entrant.sailNos:
                if sailNo.upper() not in self.sailNos:
                    self.sailNos[sailNo.upper()] = entrant
                else:
                    raise ValueError('Duplicate sail number \"{}\"'.format(sailNo))
        

    def indexGpsUnits(self):
        '''Create GPS indices for entrants'''

        for entrantId in self.entrants:
            entrant = self.entrants[entrantId]

            for gpsId in entrant.gpsIds:
                if gpsId.upper() not in self.gpsIds:
                    self.gpsIds[gpsId.upper()] = entrant
                else:
                    raise ValueError('Duplicate GPS ID \"{}\"'.format(gpsId))
        

    def loadSession(self, configPath, dataPath, dataType):
        '''Load a single event session - absence of config will result in an exception (intentional)'''
        
        session = Session(self, configPath, dataPath, dataType, verbosity=self.verbosity)

        session.loadRuns()

        self.sessions[session.date] = session

        return session


    def populateRuns(self):
        '''Populate runs from session objects'''
        
        for courseId in self.sessions:
            runs = self.sessions[courseId].runs

            for entrantId in runs:
                if entrantId in self.runs:
                    self.runs[entrantId] += runs[entrantId]
                else:
                    self.runs[entrantId] = runs[entrantId].copy()

                self.numRuns += len(runs[entrantId])


    def checkBests(self, session):
        '''Check if anyone has improved their PBs'''

        # Determine the number of decimals
        if 'Decimals' in self.eventConfig:
            decimals = self.eventConfig['Decimals']
        else:
            decimals = 2

        # Process all entrants in the current event
        for entrantId in self.entrants:

            # Get entrant, name and craft
            entrant = self.entrants[entrantId]
            name = entrant.getName()
            craft = entrant.getCraftType()

            # This safety check is required because of joint names in very early years
            if name in self.existingNames:

                # Should only consider entrant if they have completed some runs
                if entrantId in session.runs:

                    # Determine best speed of the session
                    sessionBest = round(session.runs[entrantId][0].speed, decimals)

                    # Global name spanning all years
                    existingName = self.existingNames[name]
                    existingBests = existingName.bests

                    # Check all-time best
                    if craft in existingBests:
                        existingBest = existingBests[craft]
                    else:
                        existingBest = 0

                    if sessionBest > existingBest:
                        if decimals == 3:
                            formattedBest = '{:.3f}'.format(sessionBest)
                        else:
                            formattedBest = '{:.2f}'.format(sessionBest)

                        entrant.entrantDict[T_PB] = formattedBest
                        existingBests[craft] = sessionBest

                        # Only report PBs for the current event
                        if existingBest > 0:
                            self.logInfo('New PB for {} ({}) {:.2f} -> {:.2f}'.format(name, craft, existingBest, sessionBest))


    def loadSessions(self):
        '''Load all of the event sessions'''
        
        configPaths = sorted(glob.glob(os.path.join(self.path, SESSIONS_DIR, '[1-2][0-9][0-9][0-9][0-1][0-9][0-3][0-9]')))
        for configPath in configPaths:
            found = False

            for dataType in [RUNDATA_DIR, GPSDATA_DIR]:
                dataPath = os.path.join(self.path, dataType, os.path.basename(configPath))
                if os.path.exists(dataPath):
                    session = self.loadSession(configPath, dataPath, dataType)
                    self.checkBests(session)
                    found = True

            # If no data was found then create a config-only session
            if not found:
                self.loadSession(configPath, None, None)

        self.populateRuns()
        self.sortRuns()


    def checkEntryTypes(self):
        '''Check for people who have exceeded their entry type'''

        for entrantId in self.entrants:
            entrant = self.entrants[entrantId]
            numSessions = 0
            numWeekDaySessions = 0

            for sessionId in self.sessions:
                session = self.sessions[sessionId]
                
                if entrantId in session.runs:
                    numSessions += 1
                    numWeekDaySessions += 1 if session.weekend == 'N' else 0

            entryType = entrant.getValue('Entry Type')

            if (entryType[:1] == '1' and numSessions > 1 or
                    entryType[:1] == '2' and numSessions > 2):
                self.logInfo('Entry type exceeded by {} ({}, {}) in {} - {} sessions'.format(
                    entrant.getName(), entrant.getCraftType(), entryType, self.year, numSessions))


    def initEvent(self):
        '''Load entrants and generate all of the indices'''

        self.loadConfig()
        self.loadMotions()
        self.loadEntrants()
        self.loadReports()
        self.indexNames()
        self.indexSails()
        self.indexGpsUnits()
        
        self.initialised = True
        

    def processEvent(self, runReports=True):
        '''Read entrants from config folder'''

        if self.verbosity >= 1:
            print('Processing {}...'.format(self.year))

        if not self.initialised:
            self.initEvent()

        self.loadSessions()
        self.checkEntryTypes()

        if runReports:
            for session in self.sessions.values():
                session.runReports()
            self.runReports()

        if self.verbosity >= 1:
            print('All done!\n')


    def loadResults(self):
        '''Load all of the published results'''
        
        for dataDir in [RESULTS_DIR]:
            dataPaths = sorted(glob.glob(os.path.join(self.path, dataDir, '[1-2][0-9][0-9][0-9][0-1][0-9][0-3][0-9]')))   
            for dataPath in dataPaths:
                configPath = os.path.join(self.path, SESSIONS_DIR, os.path.basename(dataPath))
                self.loadSession(configPath, dataPath, dataDir)

        self.sortRuns()

## Results Check

Standalone function to check that calculated results match published results

In [3]:
def compareResults(actualSession, expectedSession, actualResult, expectedResult):
    '''Compare expected vs actual results for a single session'''

    actualSpeed = actualResult.data[T_SPEED]
    expectedSpeed = expectedResult.data[T_SPEED]

    if actualSpeed != expectedSpeed:
        if int(actualSession.year) >= 2000 and int(actualSession.year) <= 2004:
            # Handle minor rounding errors in historical results, taking account of floating point artefacts
            if actualSpeed != math.trunc(round((expectedSpeed + 0.01) * 1000) / 10) / 100:
                actualSession.logError('Different session results for {} on {} - {} vs {}'.format(
                    actualResult.entrant.getName(), actualSession.date, actualSpeed, expectedSpeed))
        else:
            actualSession.logError('Different session results for {} on {} - {} vs {}'.format(
                actualResult.entrant.getName(), actualSession.date, actualSpeed, expectedSpeed))

    # No need to check names on the actual results because that is done when they are loaded into memory
    if actualResult.entrant.getName() != expectedResult.entrant.getName():
        actualSession.logError('Name mismatch on {} - {} vs {}'.format(
            actualSession.date, actualResult.entrant.getName(), expectedResult.entrant.getName()))

    # Run times are only present on session results for 2006, 2007 and 2009
    for timeField in ['Time', 'Start Time', 'Finish Time']:
        if timeField in actualResult.data and timeField in expectedResult.data:
            actualTime = actualResult.data[timeField]
            expectedTime = actualResult.data[timeField]
            if actualTime != expectedTime:
                actualSession.logError('Different {} for {} on {} - {} vs {}'.format(
                    timeField.lower(), actualResult.entrant.getName(), actualSession.date, actualTime, expectedTime))

    if 'Craft Type' in actualResult.data and actualResult.data['Craft Type']:
        resultCraftType = actualResult.data['Craft Type'].replace(' ', '').lower()
        entrantCraftType = actualResult.entrant.getValue('Craft Type').replace(' ', '').lower()
        if resultCraftType != entrantCraftType:
            actualSession.logError('Craft type mismatch for {} on {} - {} vs {}'.format(
                actualResult.entrant.getName(), actualSession.date,
                actualResult.data['Craft Type'], actualResult.entrant.getValue('Craft Type')))

    if 'Craft Type' in expectedResult.data and expectedResult.data['Craft Type']:
        resultCraftType = expectedResult.data['Craft Type'].replace(' ', '').lower()
        entrantCraftType = expectedResult.entrant.getValue('Craft Type').replace(' ', '').lower()
        if resultCraftType != entrantCraftType:
            actualSession.logError('Craft type mismatch for {} on {} - {} vs {}'.format(
                expectedResult.entrant.getName(), expectedSession.date,
                expectedResult.data['Craft Type'], expectedResult.entrant.getValue('Craft Type')))


def compareSessions(actualSession, expectedSession):
    '''Compare expected vs actual results for a single session'''

    if len(actualSession.runs) != len(expectedSession.runs):
        expectedSession.logError('Number of entrants does not match on {} - {} (actual) vs {} (expected)'.format(
            expectedSession.date, len(actualSession.runs), len(expectedSession.runs)))

    for entrantId in expectedSession.runs:
        if entrantId not in actualSession.runs:
            expectedSession.logWarning('Missing result for {} on {} - {} knots'.format(
                expectedSession.entrants[entrantId].getName(), expectedSession.date,
                expectedSession.runs[entrantId][0].data[T_SPEED]))           

    for entrantId in actualSession.runs:
        if entrantId not in expectedSession.runs:
            actualSession.logError('Unexpected result for {} on {} - {} knots'.format(
                actualSession.entrants[entrantId].getName(), actualSession.date,
                actualSession.runs[entrantId][0].data[T_SPEED]))           
        else:
            compareResults(actualSession, expectedSession, 
                           actualSession.runs[entrantId][0], expectedSession.runs[entrantId][0])


class TestResults(unittest.TestCase):
    '''Class to test Event class'''
    
    def testResults(self):
        '''Test event results match what was published on the website at the time'''

        verbosity = 0

        for eventYear in range(1998, 2010):
            if verbosity >= 1:
                print('Testing {}...'.format(eventYear))

            eventPath = os.path.join(projdir, EVENTS_DIR, str(eventYear))

            actual = Event(eventPath, appConfig)
            actual.initEvent()
            actual.loadSessions()

            expected = Event(eventPath, appConfig)
            expected.initEvent()
            expected.loadResults()

            for sessionDate in expected.sessions:
                actualSession = actual.sessions[sessionDate]
                expectedSession = expected.sessions[sessionDate]
                compareSessions(actualSession, expectedSession)

            if verbosity >= 1:
                print('All done!\n')


class TestReports():
    '''Class to test Event class'''
    
    def testReports(self):
        '''Test event reports match what was published on the website at the time'''

        verbosity = 1

        for eventYear in range(1998, 2022):
            if verbosity >= 1:
                print('Reporting {}...'.format(eventYear))

            eventPath = os.path.join(projdir, EVENTS_DIR, str(eventYear))

            if os.path.exists(eventPath):
                event = Event(eventPath, appConfig)
                event.initEvent()
                event.loadSessions()

                for sessionId in event.sessions:
                    session = event.sessions[sessionId]
                    session.runReports()
                event.runReports()

                if verbosity >= 1:
                    print('All done!\n')

## Unit Tests

A handful of very basic tests

In [4]:
class TestEvent(unittest.TestCase):
    '''Class to test Event class'''
    
    def testLoadEntrants(self):
        '''Test using 2003 data'''

        eventYear = '2003'
        eventPath = os.path.join(projdir, EVENTS_DIR, eventYear)

        # Vebosity is zero to suppress 'WARNING: Unrecognised GPS ID' 
        event = Event(eventPath, appConfig, verbosity=0)

        event.loadConfig()
        event.loadEntrants()
        
        # Check the entrants = 35 in entrants.csv
        self.assertEqual(len(event.entrants), 35)


    def testIndexNames(self):
        '''Test using 2003 data'''

        eventYear = '2003'
        eventPath = os.path.join(projdir, EVENTS_DIR, eventYear)

        event = Event(eventPath, appConfig)

        event.loadConfig()
        event.loadEntrants()
        event.indexNames()
        
        # Check the names = 34 in entrants.csv (2 craft types for Richard Trubger)
        self.assertEqual(len(event.names), 34)


    def testIndexSails(self):
        '''Test using 2003 data'''

        eventYear = '2003'
        eventPath = os.path.join(projdir, EVENTS_DIR, eventYear)

        event = Event(eventPath, appConfig)

        event.loadConfig()
        event.loadEntrants()
        event.indexSails()
        
        # Check the sail numbers = 36 in entrants.csv (2 sail numbers for Torix)
        self.assertEqual(len(event.sailNos), 36)


    def testIndexGpsUnits(self):
        '''Test using 2019 data'''

        eventYear = '2019'
        eventPath = os.path.join(projdir, EVENTS_DIR, eventYear)

        event = Event(eventPath, appConfig)

        event.loadConfig()
        event.loadEntrants()
        event.indexGpsUnits()
        
        # Check the entrants = 68 in entrants.csv
        self.assertEqual(len(event.gpsIds), 72)


    def testProcessEvent(self):
        '''Test using 2019 data'''

        eventYear = '2019'
        eventPath = os.path.join(projdir, EVENTS_DIR, eventYear)

        # Vebosity is zero to suppress 'WARNING: Unrecognised GPS ID' 
        event = Event(eventPath, appConfig, verbosity=0)

        event.processEvent()
        
        # Check the entrants = 68 in entrants.csv and 4 unrecognised GPS
        self.assertEqual(len(event.entrants), 68 + 4)

        # Check the event totals - 21 duplicate runs should have been removed
        self.assertEqual(event.numRuns, 3731 - 21)
        self.assertEqual(len(event.runs), 68)
        
        # Check runs are sorted correctly
        for personId in event.runs:
            maxSpeed = 99.999
            for run in event.runs[personId]:
                self.assertEqual(run.data[T_SPEED] <= maxSpeed, True)
                maxSpeed = run.data[T_SPEED]

## Run Unit Tests

Note: Only run unit tests when running this script directly, not during an import

In [5]:
if __name__ == '__main__':
    # Read main config into global variable
    filename = os.path.join(projdir, CONFIG_DIR, APP_CONFIG)
    with open(filename, 'r', encoding='utf-8') as f:
        jsonTxt = f.read()
        appConfig = json.loads(jsonTxt)

In [6]:
if __name__ == '__main__':
       unittest.main(argv=['first-arg-is-ignored'], exit=testExit)

......
----------------------------------------------------------------------
Ran 6 tests in 0.756s

OK


## All Done!